In [1]:
from ML4transients.training import PytorchDataset, CustomCNN, get_trainer, get_loss_function

2025-12-09 14:46:22.492052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765287982.515792 2134453 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765287982.523086 2134453 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765287982.542676 2134453 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765287982.542695 2134453 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765287982.542698 2134453 computation_placer.cc:177] computation placer alr

In [3]:
datasets = PytorchDataset.create_splits(
    '/sps/lsst/groups/transients/HSC/fouchez/raphael/data/rc2_coadd', 
    cutout_types=['diff', 'coadd'],  # Use both diff and coadd as 2-channel input
    random_state=42
)

Discovering data files...
Data discovery completed in 0.00s: 40 cutout visits, 40 feature visits, 0 lightcurve paths
Building sample index...
Creating splits from 57903 samples...
Loading 40531 cutouts with types ['diff', 'coadd']...
Loading 5791 cutouts with types ['diff', 'coadd']...
Loading 11581 cutouts with types ['diff', 'coadd']...


In [4]:
train_dataset = datasets['train'] 
test_dataset = datasets['test'] 
val_dataset = datasets['val'] 



In [ ]:
from torch.utils.data import DataLoader

config = {
    "epochs": 2,
    "learning_rate": 0.001,
    "batch_size": 128,
    "num_iter_per_epoch": 400,
    "epoch_decay_start": 80,
    "num_workers": 0,
    "output_dir":"../saved/training_test",
    "model_params": {
        "input_shape": [30, 30, 2],  # Changed: 2 channels (diff + coadd)
        "num_classes": 2,
        "filters_1": 32,
        "filters_2": 64,
        "dropout_1": 0.25,
        "dropout_2": 0.25,
        "dropout_3": 0.5,
        "units": 128,
        "in_channels": 2  # Added: explicitly set 2 input channels
    }
   
}

In [6]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=config['batch_size'],  # 128 from your config
    shuffle=True,
    num_workers=config['num_workers']  # 4 from your config
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=config['batch_size'], 
    shuffle=False,
    num_workers=config['num_workers']
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=config['batch_size'], 
    shuffle=False,
    num_workers=config['num_workers']
)
trainer = get_trainer("standard", config)




TensorBoard logging to: runs/experiment
Run 'tensorboard --logdir=runs' to view logs
Building CNN with 2 input channels


In [7]:
trainer.train_one_epoch(1, train_loader)

{'loss': 0.27297796153871795, 'accuracy': 0.8842367570501591}

In [8]:
best_acc = trainer.fit(train_loader, test_loader, val_loader)


Epoch 1/2 - Train: {'loss': 0.2651501960746873, 'accuracy': 0.921590881054008} - Test: {'accuracy': 0.9369711621481609, 'loss': 0.15338689364168956} - Val: {'accuracy': 0.9335117865469303, 'loss': 0.47837488705312814}


RuntimeError: Parent directory ../saved/training_test does not exist.

In [8]:
import torch
trainer = get_trainer("standard", config)
trainer.model.load_state_dict(torch.load('/sps/lsst/groups/transients/HSC/fouchez/raphael/training/simple_run/model_best.pth', map_location=torch.device('cpu')))



TensorBoard logging to: runs/experiment
Run 'tensorboard --logdir=runs' to view logs


<All keys matched successfully>

In [9]:
from torch.utils.data import DataLoader

inference_loader = DataLoader(
    val_dataset,  # Replace with your dataset
    batch_size=128,  # Batch size of 1 for inference
    shuffle=False,
    num_workers=0
)

In [10]:
from ML4transients.evaluation import infer 
result = infer(inference_loader, trainer= trainer, return_preds=True, compute_metrics=True)

In [11]:
print(result["accuracy"])
print(result["confusion_matrix"][0][0])
print(result["confusion_matrix"][0][1])
print(result["confusion_matrix"][1][0])
print(result["confusion_matrix"][1][1])

0.9528578829217752
0.7475392850975652
0.025902262130892764
0.021239854947332066
0.20531859782420997
